# Recommendation System using DA 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from heapq import nlargest

## Data Generation

Following the same procedure as Tomita et.al, 2023, I generate data as the following:

$$|{\cal J}|=n$$
$$|{\cal C}|=1.5n$$
$$p_{c,j}=\alpha\bar p_j+(1-\alpha)\tilde p_{c,j}$$
$$p_{j,c}=\alpha\bar p_c+(1-\alpha)\tilde p_{j,c}$$
where $n\in\{50,100,150,200\}$ and $\alpha\in\{0,0.25,0.5,0.75,1\}$

We rank the employers and job candidates in the following way: $j_1,j_2,...j_{|\cal J|}$ and $c_1,c_2,...c_{|\cal C|}$ such that a lower index implies a more popular user. And

$$\bar p_{j_k}=1-\frac{k-1}{|{\cal J}|-1}$$
$$\bar p_{c_k}=1-\frac{k-1}{|{\cal C}|-1}$$

Also, $\tilde p_{c,j}, \tilde p_{j,c}\sim U[0,1]$.


In [2]:
# Data Generation

np.random.seed(314)

n = 50
alpha = 0.5

bar_p_c = list(range(int(1.5 * n)))
for i in range(int(1.5 * n)):
        bar_p_c[i] = 1 - (i) / (1.5 * n - 1)
        
bar_p_j = list(range(int(n)))
for i in range(int(n)):
        bar_p_j[i] = 1 - (i) / (n - 1)
        
tilde_p_c_j = np.random.uniform(0,1,(int(1.5 * n), n))
tilde_p_j_c = np.random.uniform(0,1,(n, int(1.5 * n)))

p_c_j = np.zeros((int(1.5 * n), n))
p_j_c = np.zeros((n, int(1.5 * n)))

for i in range(int(1.5 * n)):
    for j in range(int(n)):
        p_c_j[i][j] = alpha * bar_p_j[j] + (1 - alpha) * tilde_p_c_j[i][j]
        
for i in range(int(n)):
    for j in range(int(1.5 * n)):
        p_j_c[i][j] = alpha * bar_p_c[j] + (1 - alpha) * tilde_p_j_c[i][j]
        
        
candidate_preference = pd.DataFrame(p_c_j)
job_preference = pd.DataFrame(p_j_c)

## Deferred Acceptance Algorithm

 This recommendation system is based on the deferred acceptance algorithm. The recommendation system then works in the following way: 

Step 1: each job candidate $c$ proposes to the top $n$ jobs according to the preference score $p_{c,j}$. Each firm $j$ ranks the job candidates proposing to it according to the preference score $p_{j,c}$ and reject those ranks $m+1$ or lower. 

Step $k$: each job candidate $c$ rejected by $l\leq n$ firms in Step $k-1$ proposes to $l$ firms that are ranked the highest among the firms that he/she has not proposed to yet according to $p_{c,j}$. Each firm $j$ ranks the job candidates proposing to it and job candidates who are not rejected in $k-1$ according to the preference score $p_{j,c}$ and reject those ranks $m+1$ or lower.


In [3]:
# Initialization

a = 10
b = 10

# candidates propose to jobs

have_not_propose = list(range(len(candidate_preference)))
propose_to = list(range(len(candidate_preference)))

for i in range(len(have_not_propose)):
    have_not_propose[i] = dict(zip(range(len(candidate_preference.iloc[i])),candidate_preference.iloc[i]))    
    propose_to[i] = nlargest(a, have_not_propose[i], key=have_not_propose[i].get)
    
    for k in propose_to[i]:
        have_not_propose[i].pop(k, None)
        
# jobs receive proposals
engage = list(range(len(job_preference)))

for i in range(len(engage)):
    engage[i] = []
    
    for j in range(len(propose_to)):
        if i in propose_to[j]:
            engage[i].append(j)

# jobs rank proposals
engage_value = list(range(len(engage)))

for i in range(len(engage)):
    engage_value[i] = list(range(len(engage[i])))
    
    for j in range(len(engage[i])):
        engage_value[i][j] = job_preference[engage[i][j]][i]

# jobs reject some proposals if there are more than b proposals
current_engage = list(range(len(engage)))
keep = list(range(len(engage)))
reject = list(range(len(job_preference)))

for i in range(len(current_engage)):
    current_engage[i] = dict(zip(engage[i],engage_value[i])) 
    keep[i] = nlargest(b, current_engage[i], key=current_engage[i].get)
    reject[i] = list(set(engage[i]) - set(keep[i]))

# candidates count how many jobs reject them
reject_num = list(range(int(1.5 * n)))
for i in range(len(reject_num)):
    reject_num[i] = 0
    for j in range(len(reject)):
        if i in reject[j]:
            reject_num[i] = reject_num[i] + 1

In [4]:
# loop
step = 0

while max(reject_num) != 0:
    
    #print(step, reject_num)
    #step = step + 1
    
    # candidates propose to jobs
    propose_to = list(range(len(candidate_preference)))

    for i in range(len(propose_to)):
        propose_to[i] = nlargest(reject_num[i], have_not_propose[i], key=have_not_propose[i].get)

        for k in propose_to[i]:
            have_not_propose[i].pop(k, None)

    # jobs receive proposals
    engage = keep

    for i in range(len(engage)):    
        for j in range(len(propose_to)):
            if i in propose_to[j]:
                engage[i].append(j)


    # jobs rank proposals
    engage_value = list(range(len(engage)))

    for i in range(len(engage)):
        engage_value[i] = list(range(len(engage[i])))

        for j in range(len(engage[i])):
            engage_value[i][j] = job_preference[engage[i][j]][i]

    # jobs reject some proposals if there are more than b proposals
    current_engage = list(range(len(engage)))
    keep = list(range(len(engage)))
    reject = list(range(len(job_preference)))

    for i in range(len(current_engage)):
        current_engage[i] = dict(zip(engage[i],engage_value[i])) 
        keep[i] = nlargest(b, current_engage[i], key=current_engage[i].get)
        reject[i] = list(set(engage[i]) - set(keep[i]))

    # candidates count how many jobs reject them
    reject_num = list(range(int(1.5 * n)))
    for i in range(len(reject_num)):
        reject_num[i] = 0
        for j in range(len(reject)):
            if i in reject[j]:
                reject_num[i] = reject_num[i] + 1

## Recommendation

The following table shows the resulting recommendation. The row indexes represent firms. Each row shows the candidates this firm will be recommended to. For example, firm $0$ is recommended to candidate $2,3,5,20,23,13,25,17,15,0$.

In [6]:
recommend = pd.DataFrame(keep)
recommend

,0,1,2,3,4,5,6,7,8,9
0,2,3,5,20,23,13,25,17,15,0
1,6,13,14,7,25,1,16,30,9,31
2,0,6,19,11,9,13,23,33,2,27
3,0,1,5,7,8,13,15,24,6,25
4,15,6,10,8,18,0,7,36,48,4
5,8,11,17,15,9,24,18,1,2,12
6,11,8,17,0,4,32,29,1,26,45
7,5,11,3,1,23,17,27,9,7,24
8,0,13,15,18,27,30,22,5,21,16
9,19,2,34,10,5,24,30,16,29,37
